<a href="https://colab.research.google.com/github/konstantine25b/Neural_Network_ML_Facial_Expression_Recognition_Challenge/blob/main/Facial_Expression_Recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lets goo Facial Expression Recognition